In [1]:
from collections import defaultdict
import json

In [2]:
user_to_location = defaultdict(set)

# adding different datasets records into the user_to_location dictionary
for dataset_file in ['TREC2016/Phase1_requests.json', 'TREC2015/batch_requests.json']:
	with open(dataset_file) as f:
		for line in f:
			response = json.loads(line)
			user_id = response['body']['person']['id']
			for preference in response['body']['person']['preferences']:
				user_to_location[user_id].add((preference['documentId'], preference['rating']))

In [3]:
# split data per user interaction w.r.t split ratio
split_ratio = 0.7

data_all = open('TREC_1516_All.txt', 'w')
data_train = open('TREC_1516_Train.txt', 'w')
data_test = open('TREC_1516_Test.txt', 'w')

for uid, lids_ratings in user_to_location.items():
    counter = 0
    train_rate = len(lids_ratings) * split_ratio
    for lid, rating in lids_ratings:
        data_all.write(f"{uid}\t{lid}\t{rating}\n")
        if counter >= train_rate:
            data_test.write(f"{uid}\t{lid}\t{rating}\n")
            counter += 1
        else:
            data_train.write(f"{uid}\t{lid}\t{rating}\n")
            counter += 1

data_all.close()
data_train.close()
data_test.close()